In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from jax_unirep.utils import get_embeddings
from jax_unirep.params import add_dense_params
from jax_unirep.layers import mlstm1900, dense
from jax_unirep.activations import softmax, identity
from jax_unirep.losses import neg_cross_entropy_loss
from jax_unirep.utils import aa_seq_to_int, aa_to_int, load_params_1900


import numpy as np


from functools import partial
from jax import grad, jit
from jax.experimental.optimizers import adam
from typing import List, Dict

/usr/local/Caskroom/miniconda/base/envs/jax-unirep/lib/python3.7/site-packages/jax/lib/xla_bridge.py:119: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [2]:
oh_arrs = np.eye(len(aa_to_int))
one_hots = {aa_to_int[k]: oh_arrs[i] for i, k in enumerate(aa_to_int.keys())}

In [3]:
sequences = [
    "HASTA",
    "HASTH",
    "VISTA",
]
next_letters = [s[1:] for s in sequences]
sequences_int = [aa_seq_to_int(s) for s in sequences]
next_letters_int = [aa_seq_to_int(s) for s in next_letters]
embeddings = np.load("jax_unirep/weights/1900_weights/UniRef50/embed_matrix:0.npy")
x = np.stack([embeddings[i] for i in sequences_int], axis=0)[:, :-1, :]
y = []
for seq in next_letters_int:
    next_letter_arr = np.stack([one_hots[i] for i in seq])
    y.append(next_letter_arr)
y = np.stack(y)
print(x.shape, y.shape)

(3, 6, 10) (3, 6, 28)


In [4]:
params = dict()
params["mlstm1900"] = load_params_1900()
params = add_dense_params(params, "dense", 1900, 28)

In [6]:
def predict(params, batch):
    _, _, batch = mlstm1900(params["mlstm1900"], batch)
    batch = dense(params["dense"], batch, activation=softmax)
    return batch

In [107]:
y_hat = predict(params, x)

In [106]:
loss(params=params, x=x, y=y)

DeviceArray(nan, dtype=float32)

In [109]:
xent = -(y * np.log(y_hat + 1E-10) + (1 - y) * np.log((1 - y_hat) + 1E-10))
# np.mean(xent)

/usr/local/Caskroom/miniconda/base/envs/jax-unirep/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.


In [ ]:
params

In [7]:
loss = partial(neg_cross_entropy_loss, model=predict)
dloss = jit(grad(loss))

init, update, get_params = adam(step_size=0.005)

state = init(params)

for i in range(20):
    print(i)
    g = dloss(params, x=x, y=y)

    state = update(i, g, state)
    params = get_params(state)

#     if i % 10 == 0:
    l = loss(params, x=x, y=y)
    if np.isnan(l):
        break
    print(f"iteration: {i}, loss: {l}")
# def update(params, x, y):

# prepare the sequences
# batch sequences according to length

0
iteration: 0, loss: 0.08237417042255402
1
iteration: 1, loss: 0.04163924977183342
2
iteration: 2, loss: 0.054564714431762695
3
iteration: 3, loss: 0.022856971248984337
4
iteration: 4, loss: 0.01080869697034359
5


In [ ]:
params["dense"]["b"].shape

In [36]:
batch.shape

(3, 239, 1900)

In [37]:
def softmax_noaxis(x):
    print(x.shape)
    return np.exp(x) / np.sum(np.exp(x))

def softmax_newaxis(x):
    e = np.exp(batch)
    return np.divide(e, np.sum(e, axis=-1)[:, :, None])

In [39]:
_, _, batch = mlstm1900(params["mlstm1900"], x)
batch = dense(params["dense"], batch, activation=identity)

sfm_noaxis = softmax_noaxis(batch)
sfm_newaxis = softmax_newaxis(batch)

(3, 239, 28)


In [ ]:
original = np.exp() / np.sum(np.exp(x))

In [32]:
batch.shape

(3, 239, 28)